# Use Obspy to manage inventory XML files

## Fei Zhang
## created on 2020-09-18

- split a big xml file into multiple single-station xml files

- modify each station xml file with new GA-metadata

- concatenate the mutiple station xml files into one big xml file

In [1]:
import os
import obspy
import obspy
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL


In [2]:
print (obspy.__version__)

1.2.1


In [3]:
BIG_INVENTOTOR_FILE="/Datasets/InventoryXml/OA_stations_2017-2018.xml"

DIR4XML="/Datasets/InventoryXml/OA_stations_2017-2018"



In [4]:
if not os.path.exists(DIR4XML):
    os.mkdir(DIR4XML)

In [5]:

big_inv = obspy.read_inventory(BIG_INVENTOTOR_FILE) 


In [6]:
type(big_inv.networks)

list

In [7]:
for net in big_inv.networks:
    print(len(net.stations))
    print(net.stations[0].code)
    print(net.stations[-1].code)

328
BS24
CJ27


In [8]:
for a_net in big_inv.networks:
    for a_sta in a_net.stations:
        
        print(a_net.code, a_sta.code)
        
        a_inv = big_inv.select(network=a_net.code, station=a_sta.code)  #.copy()
        
        # modify station metadata
#         my_sensor=obspy.core.inventory.util.Equipment(type="Sensor", description="Nanometrics Trillium Compact 120s",serial_number="004940")
#         my_digitizer = obspy.core.inventory.util.Equipment(type="Digitizer", description="Guralp Minimus",serial_number="MIN-A456")

#         a_sta.equipments = [my_sensor, my_digitizer]
        
        sta_file_name = "%s_%s_station.xml"%(a_net.code, a_sta.code)
        
        outxml = os.path.join(DIR4XML, sta_file_name)
        a_inv.write(outxml,format="stationxml") # nsmap={'GeoscienceAustralia': GA_NameSpace})
        # the xml file

    

OA BS24
OA BS25
OA BS26
OA BS27
OA BS28
OA BT23
OA BT24
OA BT25
OA BT26
OA BT27
OA BT28
OA BU22
OA BU23
OA BU24
OA BU25
OA BU26
OA BU27
OA BU28
OA BV21
OA BV22
OA BV23
OA BV24
OA BV26
OA BV27
OA BV28
OA BW20
OA BW21
OA BW22
OA BW23
OA BW24
OA BW25
OA BW26
OA BW27
OA BW28
OA BX20
OA BX21
OA BX22
OA BX23
OA BX24
OA BX25
OA BX26
OA BX27
OA BX28
OA BY20
OA BY21
OA BY22
OA BY23
OA BY24
OA BY25
OA BY26
OA BY27
OA BY28
OA BZ20
OA BZ21
OA BZ22
OA BZ23
OA BZ24
OA BZ25
OA BZ26
OA BZ27
OA BZ28
OA CA20
OA CA21
OA CA22
OA CA23
OA CA24
OA CA25
OA CA26
OA CA27
OA CA28
OA CB20
OA CB21
OA CB22
OA CB23
OA CB24
OA CB25
OA CB26
OA CB27
OA CB28
OA CC20
OA CC21
OA CC22
OA CC23
OA CC24
OA CC25
OA CC26
OA CC27
OA CC28
OA CD21
OA CD22
OA CD23
OA CD24
OA CD25
OA CD26
OA CD27
OA CD28
OA CE22
OA CE23
OA CE24
OA CE25
OA CE26
OA CE27
OA CE28
OA CF22
OA CF23
OA CF24
OA CF25
OA CF27
OA CF28
OA CG21
OA CG22
OA CG23
OA CH21
OA CH22
OA CH23
OA CI21
OA CI22
OA CI23
OA CJ27
OA BS24
OA BS25
OA BS27
OA BS28
OA BT23
OA BT24


# Modify the existing station XML files inclding new metadata

- add sensor digitizer
- add extra metadata: GPS correction
- add extra metadata: Orientation correction

In [9]:
# Construct a new inventory object of networks.
# This will use new obspy version and new attributes:
inv2 = Inventory(
    # We'll add networks later.
    networks=[],
    # The source should be the id whoever create the file.
    source="Geoscience Australia EFTF AusArray")


In [10]:
for a_net in big_inv.networks:
    
    # Re-write each network of the big inventory into the new object inv2 
    inv2.networks = []  
    inv2.networks.append(a_net)
    inv2.write(a_net.code+"_stations.xml", format="stationxml", validate=True)  # every Station got equipment
    
    print(len(a_net.stations))
    
    for a_sta in a_net.stations:
        # print(a_net.code, a_sta.code)
        
        a_inv = big_inv.select(station=a_sta.code).copy() #may be necessary!!!
        
        print ("Number of stations= ", len(a_sta), a_sta.code)
        
        #for each_sta in a_sta:  # ?? into channels
        # modify station metadata
        my_sensor=obspy.core.inventory.util.Equipment(type="Sensor", description="Nanometrics Trillium Compact 120s",serial_number="004940")
        my_digitizer = obspy.core.inventory.util.Equipment(type="Digitizer", description="Guralp Minimus",serial_number="MIN-A456")

        a_sta.equipments = [my_sensor, my_digitizer]
        
#         sta_file_name = "%s_%s_station.xml"%(a_net.code, a_sta.code)
#         outxml = os.path.join(DIR4XML, sta_file_name)      
#         a_inv.write(outxml,format="stationxml") # nsmap={'GeoscienceAustralia': GA_NameSpace})
        
        sta_file_name2 = "%s_%s_station2.xml"%(a_net.code, a_sta.code)
        outxml2 = os.path.join(DIR4XML, sta_file_name2)

        inv2.networks = a_inv.networks

        inv2.write(outxml2,format="stationxml", validate=True) # nsmap={'GeoscienceAustralia': GA_NameSpace})

    # the big inventory in new object inv2 
    inv2.networks = []  
    inv2.networks.append(a_net)
    inv2.write(a_net.code+"_stations2.xml", format="stationxml", validate=True)  # every Station got equipment

328
Number of stations=  3 BS24
Number of stations=  3 BS25
Number of stations=  3 BS26
Number of stations=  3 BS27
Number of stations=  3 BS28
Number of stations=  3 BT23
Number of stations=  3 BT24
Number of stations=  3 BT25
Number of stations=  3 BT26
Number of stations=  3 BT27
Number of stations=  3 BT28
Number of stations=  3 BU22
Number of stations=  3 BU23
Number of stations=  3 BU24
Number of stations=  3 BU25
Number of stations=  3 BU26
Number of stations=  3 BU27
Number of stations=  3 BU28
Number of stations=  3 BV21
Number of stations=  3 BV22
Number of stations=  3 BV23
Number of stations=  3 BV24
Number of stations=  3 BV26
Number of stations=  3 BV27
Number of stations=  3 BV28
Number of stations=  3 BW20
Number of stations=  3 BW21
Number of stations=  3 BW22
Number of stations=  3 BW23
Number of stations=  3 BW24
Number of stations=  3 BW25
Number of stations=  3 BW26
Number of stations=  3 BW27
Number of stations=  3 BW28
Number of stations=  3 BX20
Number of statio

Number of stations=  3 CC25
Number of stations=  3 CC26
Number of stations=  3 CC27
Number of stations=  3 CC28
Number of stations=  3 CD21
Number of stations=  3 CD22
Number of stations=  3 CD23
Number of stations=  3 CD24
Number of stations=  3 CD25
Number of stations=  3 CD26
Number of stations=  3 CD27
Number of stations=  3 CD28
Number of stations=  3 CE22
Number of stations=  3 CE23
Number of stations=  3 CE24
Number of stations=  3 CE25
Number of stations=  3 CE26
Number of stations=  3 CE27
Number of stations=  3 CE28
Number of stations=  3 CF22
Number of stations=  3 CF23
Number of stations=  3 CF24
Number of stations=  3 CF25
Number of stations=  3 CF27
Number of stations=  3 CF28
Number of stations=  3 CG21
Number of stations=  3 CG22
Number of stations=  3 CG23
Number of stations=  3 CH21
Number of stations=  3 CH22
Number of stations=  3 CH23
Number of stations=  3 CI21
Number of stations=  3 CI22
Number of stations=  3 CI23
Number of stations=  3 CJ27


# TODOs:  

## for each (net, sta):
* get the real Sensor and Ditizer from OA_sensors_digitizers.csv 

* get the GPS corrections

* get the orientation corrections

###

(hiperseis) fzhang@zubuntu1804 ~/Githubz/hiperseis/seismic/inventory/sandbox (develop) 
 $ grep "Guralp Minimus"  OA_sensors_digitizers.csv | wc
    262     524   10856
(hiperseis) fzhang@zubuntu1804 ~/Githubz/hiperseis/seismic/inventory/sandbox (develop) 
 $ grep "120s"  OA_sensors_digitizers.csv | wc
    218     872   12654
(hiperseis) fzhang@zubuntu1804 ~/Githubz/hiperseis/seismic/inventory/sandbox (develop) 
 $ wc OA_sensors_digitizers.csv
  481  1397 23683 OA_sensors_digitizers.csv
(hiperseis) fzhang@zubuntu1804 ~/Githubz/hiperseis/seismic/inventory/sandbox (develop) 
 $ grep BV26 OA_sensors_digitizers.csv
33850940,OABV26,,Guralp Minimus,MIN-4556
33850940,OABV26,,Nanometrics Trillium Compact 120s,004768


# How to merge the individual xml files into One?

## use new obspy version

In [11]:
stop here

SyntaxError: invalid syntax (<ipython-input-11-a96ba3aab008>, line 1)

# How to create a new station xml file

Adapted from https://docs.obspy.org/tutorial/code_snippets/stationxml_file_from_scratch.html


In [ ]:
import obspy
from obspy.core.inventory import Inventory, Network, Station, Channel, Site
from obspy.clients.nrl import NRL


def create_new_inventory(out_file_name="new_inventory.xml"):

    # We'll first create all the various objects. These strongly follow the
    # hierarchy of StationXML files.
    inv = Inventory(
        # We'll add networks later.
        networks=[],
        # The source should be the id whoever create the file.
        source="Geoscience Australia EFTF Program AusArray")

    net = Network(
        # This is the network code according to the SEED standard.
        code="XX",
        # A list of stations. We'll add one later.
        stations=[],
        description="A test stations.",
        # Start-and end dates are optional.
        start_date=obspy.UTCDateTime(2016, 1, 2))

    sta = Station(
        # This is the station code according to the SEED standard.
        code="ABC",
        latitude=1.0,
        longitude=2.0,
        elevation=345.0,

        creation_date=obspy.UTCDateTime(2016, 1, 2), # <CreationDate>2016-01-02T00:00:00.000000Z</CreationDate>

        start_date = obspy.UTCDateTime(2016, 1, 3),
        end_date = obspy.UTCDateTime(2020, 1, 3),
        #<Station code="ABC" startDate="2016-01-03T00:00:00.000000Z" endDate="2020-01-03T00:00:00.000000Z">
        site=Site(name="First station"))

    cha = Channel(
        # This is the channel code according to the SEED standard.
        code="HHZ",
        # This is the location code according to the SEED standard.
        location_code="",
        # Note that these coordinates can differ from the station coordinates.
        latitude=1.0,
        longitude=2.0,
        elevation=345.0,
        depth=10.0,
        azimuth=0.0,
        dip=-90.0,
        sample_rate=200)

    # By default this accesses the NRL online. Offline copies of the NRL can
    # also be used instead
    nrl = NRL()
    # The contents of the NRL can be explored interactively in a Python prompt,
    # see API documentation of NRL submodule:
    # http://docs.obspy.org/packages/obspy.clients.nrl.html
    # Here we assume that the end point of data logger and sensor are already
    # known:
    response = nrl.get_response( # doctest: +SKIP
        sensor_keys=['Streckeisen', 'STS-1', '360 seconds'],
        datalogger_keys=['REF TEK', 'RT 130 & 130-SMA', '1', '200'])


    # Now tie it all together.
    cha.response = response
    sta.channels.append(cha)
    net.stations.append(sta)
    inv.networks.append(net)

    # And finally write it to a StationXML file. We also force a validation against
    # the StationXML schema to ensure it produces a valid StationXML file.
    #
    # Note that it is also possible to serialize to any of the other inventory
    # output formats ObsPy supports.
    inv.write(out_file_name, format="stationxml", validate=True)

In [ ]:
create_new_inventory()

In [ ]:
2.27*1.09

In [ ]:
2800*0.25 + 15000*0.08 + 200 + 200